This notebook manages particle tracking module. It sets up the environment, then saves the particles in HDF5 files.

In [1]:
import numpy as np
import pylab as plt
from lpa_pdb_diagnostics import *
from scipy.constants import e, c, m_e, epsilon_0
import os
import matplotlib
%matplotlib inline

#Setting up directories
Setting up directories to fetch data and the output directory for results.

In [2]:
dir_path = "/Volumes/Orsay/circ_a01.1_foc1.9mm_400um_dens7.8_nzplambda20/data/"
#dir_path = os.getcwd() + "/data/"

In [3]:
res_path = config.result_path

#Laser plasma parameters

In [4]:
# Some definitions of the laser plasma parameters
lambda0 = 0.8e-6
w0 = 2*np.pi*c/lambda0
laser_waist = 17e-6
plasma_dens = 7.8e24
plasma_length = 2.4e-3
wp = np.sqrt(plasma_dens * e**2/ (epsilon_0 * m_e))
lambda_plasma = 2*np.pi*c/wp

#Numerical parameters

In [5]:
zmax = 2*lambda0
zmin = zmax - 6*lambda_plasma
nzplambda = 20
w_size = zmax - zmin
Nz = int(w_size*nzplambda/lambda0)
dz = w_size/Nz

#Generating file array 
We choose to only do particle tracking on N6 and N7 electrons. Other electrons can also be written in the same way.

In [6]:
inf = 0
sup = 0
period_int = 1000
period_ext = 1000
val = values( inf, sup, period_int, period_ext, plasma_length/dz )
longitudinal_position = val*dz

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [7]:
N6 = np.empty(len(val),dtype="S100")
N7 = np.empty(len(val),dtype="S100")
H = np.empty(len(val),dtype="S100")

# Initialize file names
for i, v in enumerate(val):
    N6[i] = dir_path + "N6%06d.pdb" %v
    N7[i] = dir_path + "N7%06d.pdb" %v
    H[i] = dir_path + "H%06d.pdb" %v

In [8]:
debutItInstant = int(2.2e-3/dz/period_int) #round off to the nearest 1000
endItInstant = int(2.4e-3/dz/period_int) #round off to the nearest 1000

#fileArray is needed for particle tracking module for the looping
fileArray_N6 = N6[debutItInstant:endItInstant + 1]
fileArray_N7 = N7[debutItInstant:endItInstant + 1]

#Instantiating particles of the last position
We instantiate the particles of the last position and in the following cell, we choose particles of our interest based on their $\gamma$ and ROI (Region of Interest).

In [9]:
N66 = ParticleInstant(N6[ endItInstant  ])
N77 = ParticleInstant(N7[ endItInstant  ])
qdict = N66.get_qdict() ###qdict is normally the same for all particle quantities

** Processing ** Particles: Initialisation of /Volumes/Orsay/circ_a01.1_foc1.9mm_400um_dens7.8_nzplambda20/data/N6060000.pdb **
** Processing ** Particles: Initialisation of /Volumes/Orsay/circ_a01.1_foc1.9mm_400um_dens7.8_nzplambda20/data/N7060000.pdb **


In [10]:
# we choose particles which have a gamma threshold between 40 to 400 
# and situated in the first accelerating bucket 
cPN6 = N66.select( gamma_threshold = [40,200] )
cPN7 = N77.select( gamma_threshold = [40,200] )

#Running Particle Tracking 

In [11]:
PN6 = ParticleTracking( fileArray_N6, cPN6, "N6", NUM_TRACKED_PARTICLES = 2 )
PN6.run()c

A file named TrParticles_N6.hdf5 in /Users/pmlee/Documents/LPA_PDB_diagnostics/quickstart_notebook/result/ is created.
Running particle tracking algorithm. This may take a while...
** Processing ** Particles: Initialisation of /Volumes/Orsay/circ_a01.1_foc1.9mm_400um_dens7.8_nzplambda20/data/N6059000.pdb **
> /Users/pmlee/Documents/LPA_PDB_diagnostics/quickstart_notebook/build/bdist.macosx-10.5-x86_64/egg/lpa_pdb_diagnostics/particle_tracking.py(160)run()
-> if len(self.particle_buffer[ssn_dict[ssn]]) == 0:
(Pdb) reshaped_current_chosen_particles
array([[  3.21685688e+15],
       [  1.16493510e-03],
       [ -1.16807697e-05],
       [ -4.03010362e-05],
       [  2.24086711e-03],
       [ -8.37871537e-01],
       [ -3.16894711e+00],
       [  4.58386116e+00],
       [  4.96242963e+00],
       [ -2.80405721e+10],
       [ -2.80367910e+11],
       [ -1.31768814e+08],
       [ -9.05254615e+02],
       [ -9.67250751e+01],
       [ -1.26873434e+00]])
(Pdb) self.particle_buffer[ssn_dict[ssn]]

KeyboardInterrupt: 

In [ ]:
#PN7 = ParticleTracking( fileArray_N7, cPN7, "N7", NUM_TRACKED_PARTICLES = 10000 )
#PN7.run()